In [1]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from scipy.io import savemat
from scipy.interpolate import griddata  # 插值方法
import seaborn as sns  # 画图工具
from Poisson2D_model import Sampler, Poisson2D

if __name__ == '__main__':



    def u(x):
        return np.exp(-500 * ((x[:, 0:1] - 0.5)**2 + (x[:, 1:2] - 0.5)**2))+np.exp(-500 * ((x[:, 0:1] + 0.5)**2 + (x[:, 1:2] + 0.5)**2))

    def u_xx(x):
        return (1000000 * (x[:, 0:1] - 0.5)**2-1000) * np.exp(-500 * ((x[:, 0:1] - 0.5)**2 + (x[:, 1:2] - 0.5)**2))+(1000000 * (x[:, 0:1] + 0.5)**2-1000) * np.exp(-500 * ((x[:, 0:1] + 0.5)**2 + (x[:, 1:2] + 0.5)**2))

    def u_yy(x):
        return (1000000 * (x[:, 1:2] - 0.5)**2-1000) * np.exp(-500 * ((x[:, 0:1] - 0.5)**2 + (x[:, 1:2] - 0.5)**2))+(1000000 * (x[:, 1:2] + 0.5)**2-1000) * np.exp(-500 * ((x[:, 0:1] + 0.5)**2 + (x[:, 1:2] + 0.5)**2))


    # Forcing
    def f(x):
        return -u_xx(x) - u_yy(x) 


    def operator(u, x1, x2,sigma_x1=1.0, sigma_x2=1.0):
        u_x1 = tf.gradients(u, x1)[0] / sigma_x1
        u_x2 = tf.gradients(u, x2)[0] / sigma_x2
        u_xx1 = tf.gradients(u_x1, x1)[0] / sigma_x1
        u_xx2 = tf.gradients(u_x2, x2)[0] / sigma_x2
        residual = -u_xx1 - u_xx2 
        return residual


    # Parameter


    # Domain boundaries
    bc1_coords = np.array([[-1.0, -1.0],
                           [-1.0, 1.0]])
    bc2_coords = np.array([[-1.0, -1.0],
                           [1.0, -1.0]])
    bc3_coords = np.array([[1.0, -1.0],
                           [1.0, 1.0]])
    bc4_coords = np.array([[-1.0, 1.0],
                           [1.0, 1.0]])

    dom_coords = np.array([[-1.0,-1.0],
                           [1.0, 1.0]])

    # Create initial conditions samplers
    ics_sampler = None

    # Create boundary conditions samplers 创建边界条件的采样器
    bc1 = Sampler(2, bc1_coords, lambda x: u(x), name='Dirichlet BC1')
    bc2 = Sampler(2, bc2_coords, lambda x: u(x), name='Dirichlet BC2')
    bc3 = Sampler(2, bc3_coords, lambda x: u(x), name='Dirichlet BC3')
    bc4 = Sampler(2, bc4_coords, lambda x: u(x), name='Dirichlet BC4')
    bcs_sampler = [bc1, bc2, bc3, bc4]

    # Create residual sampler
    res_sampler = Sampler(2, dom_coords, lambda x: f(x), name='Forcing')
    u_sampler = Sampler(2, dom_coords, lambda x: u(x), name='solution')
    # Define model
    mode = 'M3'  # Method: 'M1', 'M2', 'M3', 'M4'
    stiff_ratio = False  # Log the eigenvalues of Hessian of losses
    dir = "Poisson2D"
    #layers = [2, 125, 200, 100, 100, 80,1]
    layers = [2, 200, 200, 200, 200, 200,1]
    #layers = [2, 125, 125, 125, 125, 125,1]
    # Test data
    nn = 256
    x1 = np.linspace(dom_coords[0, 0], dom_coords[1, 0], nn)[:, None]
    x2 = np.linspace(dom_coords[0, 1], dom_coords[1, 1], nn)[:, None]
    x1, x2 = np.meshgrid(x1, x2)
    X_star = np.hstack((x1.flatten()[:, None], x2.flatten()[:, None]))

    # Exact solution 精确解  下面都用u_star表示
    u_star = u(X_star)
    f_star = f(X_star)
    model = Poisson2D(dir, layers, operator, bcs_sampler, res_sampler, u_sampler, mode, stiff_ratio, u_star,X_star)

    # Train model
    model.train(nIter=100000, batch_size=128)
    # Predictions
    u_pred = model.predict_u(X_star)
    f_pred = model.predict_r(X_star)

    # Relative error
    error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)  # 括号中的2指的是2范数
    error_f = np.linalg.norm(f_star - f_pred, 2) / np.linalg.norm(f_star, 2)

    print('Relative L2 error_u: {:.2e}'.format(error_u))
    print('Relative L2 error_f: {:.2e}'.format(error_f))
    data_save = {}
    data_save['step'] = np.array(model.step_log).reshape(len(model.step_log), 1)
    data_save['loss'] = np.array(model.loss_log).reshape(len(model.loss_log), 1)
    # data_save['constant_bcs_val'] = np.array(model.adaptive_constant_log).reshape(
    #     len(model.adaptive_constant_log), 1)
    data_save['X'] = X_star
    data_save['err_l2r'] = np.array(model.err_l2r_log).reshape(len(model.err_l2r_log), 1)
    data_save['u_test'] = u_star
    data_save['u_pred'] = u_pred
    savemat(model.dir + '/data.mat', data_save)
    ### Plot ###

    # Exact solution & Predicted solution
    # Exact soluton
    U_star = griddata(X_star, u_star.flatten(), (x1, x2), method='cubic')  # x_star指的是point，u_star.flatten指的是对应的值
    print(U_star.shape)
    F_star = griddata(X_star, f_star.flatten(), (x1, x2), method='cubic')

    # Predicted solution
    U_pred = griddata(X_star, u_pred.flatten(), (x1, x2), method='cubic')
    F_pred = griddata(X_star, f_pred.flatten(), (x1, x2), method='cubic')

    fig_1 = plt.figure(1, figsize=(18, 5))
    plt.subplot(1, 3, 1)  # 1行3列，后面的1表示从左到右，从上到下的第1个位置
    # plt.pcolor(x1, x2, U_star, cmap='jet')  # 指的是颜色映射
    e = np.reshape(U_star, (x1.shape[0], x1.shape[1]))
    plot = plt.pcolormesh(x1, x2, e, shading='gouraud', cmap='jet', vmin=0, vmax=np.max(e))
    plt.colorbar(plot, format="%1.1e")
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.title('Exact $u(x)$')

    plt.subplot(1, 3, 2)
    e = np.reshape(U_pred, (x1.shape[0], x1.shape[1]))
    plot = plt.pcolormesh(x1, x2, e, shading='gouraud', cmap='jet', vmin=0, vmax=np.max(e))
    plt.colorbar(plot, format="%1.1e")
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.title('Predicted $u(x)$')

    plt.subplot(1, 3, 3)
    e = np.reshape(abs(U_pred-U_star), (x1.shape[0], x1.shape[1]))
    plot = plt.pcolormesh(x1, x2, e, shading='gouraud', cmap='jet', vmin=0, vmax=np.max(e))
    plt.colorbar(plot, format="%1.1e")
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.title('Absolute error')
    
    plt.savefig('Poisson2d.png', dpi=300)  # 保存为 PNG 格式，分辨率为 300 DPI

    plt.tight_layout()
    plt.show()

    # Residual loss & Boundary loss
    loss_res = model.loss_res_log
    loss_bcs = model.loss_bcs_log
    # 以下画的是figure2
    fig_2 = plt.figure(2)
    ax = fig_2.add_subplot(1, 1, 1)
    ax.plot(loss_res,'b', label='$\mathcal{L}_{r}$')
    ax.plot(loss_bcs,'r', label='$\mathcal{L}_{u_b}$')
    ax.set_yscale('log')
    ax.set_xlabel('iterations')
    ax.set_ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.grid(linestyle=":")
    plt.show()

    # Adaptive Constant
    adaptive_constant = model.adpative_constant_log

    fig_3 = plt.figure(3)
    ax = fig_3.add_subplot(1, 1, 1)
    ax.plot(adaptive_constant,'b', label='$\lambda_{u_b}$')
    ax.set_yscale('log')
    ax.set_xlabel('iterations')
    ax.set_ylabel('adaptive_constant')
    plt.legend()
    plt.tight_layout()
    plt.grid(linestyle=":")
    plt.show()

    # Gradients at the end of training
    data_gradients_res = model.dict_gradients_res_layers
    data_gradients_bcs = model.dict_gradients_bcs_layers

    gradients_res_list = []
    gradients_bcs_list = []

    num_hidden_layers = len(layers) - 1
    for j in range(num_hidden_layers):
        gradient_res = data_gradients_res['layer_' + str(j + 1)][-1]
        gradient_bcs = data_gradients_bcs['layer_' + str(j + 1)][-1]

        gradients_res_list.append(gradient_res)
        gradients_bcs_list.append(gradient_bcs)

    cnt = 1
    fig_4 = plt.figure(4, figsize=(13, 4))
    for j in range(num_hidden_layers):
        ax = plt.subplot(2, 3, cnt)
        ax.set_title('Layer {}'.format(j + 1))
        ax.set_yscale('symlog')
        gradients_res = data_gradients_res['layer_' + str(j + 1)][-1]  # 残差的梯度变化
        gradients_bcs = data_gradients_bcs['layer_' + str(j + 1)][-1]  # 边界的梯度变化
        sns.distplot(gradients_bcs, hist=False,
                     kde_kws={"shade": False},
                     # norm_hist=True, label=r'$\nabla_\theta \lambda_{u_b} \mathcal{L}_{u_b}$')
                     norm_hist = True, label = r'$\nabla_\theta \mathcal{L}_{u_b}$')
        sns.distplot(gradients_res, hist=False,
                     kde_kws={"shade": False},
                     norm_hist=True, label=r'$\nabla_\theta \mathcal{L}_r$')

        # ax.get_legend().remove()   #legend显示图中的标签
        ax.set_xlim([-3.0, 3.0])  # x轴的范围
        ax.set_ylim([0, 100])
        cnt += 1
    handles, labels = ax.get_legend_handles_labels()
    fig_4.legend(handles, labels, loc="upper left", bbox_to_anchor=(0.44, 0.09),
                 borderaxespad=0, bbox_transform=fig_4.transFigure, ncol=2)
    plt.tight_layout()
    plt.gcf().subplots_adjust(bottom=0.2)
    plt.show()

    relatie_error = model.err_l2r_log
    fig_5 = plt.figure(5)
    ax = fig_5.add_subplot(1, 1, 1)
    ax.plot(relatie_error, 'r')
    ax.set_yscale('log')
    ax.set_xlabel('iterations')
    ax.set_ylabel('relative_error')
    plt.legend()
    plt.grid(linestyle=":")
    plt.tight_layout()
    plt.show()













2025-02-22 15:28:54.920401: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 15:28:54.921285: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 15:28:54.923880: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 15:28:54.932010: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740209334.945614    1441 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740209334.94

Instructions for updating:
non-resource variables are not supported in the long term
Device mapping: no known devices.


W0000 00:00:1740209337.132552    1441 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0000 00:00:1740209340.653370    1441 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-02-22 15:29:00.681191: I tensorflow/core/common_runtime/placer.cc:162] random_normal/RandomStandardNormal: (RandomStandardNormal): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-22 15:29:00.681207: I tensorflow/core/common_runtime/placer.cc:162] random_normal/mul: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-22 15:29:00.681211: I tensorflow/core/common_runtime/placer.cc:162] random_normal: (AddV2): /job:localhost/replica:0/task:0/device:CPU:0
2025-02-22 15:29:00.681214: I tensorflow/cor

random_normal/RandomStandardNormal: (RandomStandardNormal): /job:localhost/replica:0/task:0/device:CPU:0
random_normal/mul: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
random_normal: (AddV2): /job:localhost/replica:0/task:0/device:CPU:0
mul: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
Variable: (VariableV2): /job:localhost/replica:0/task:0/device:CPU:0
Variable/Assign: (Assign): /job:localhost/replica:0/task:0/device:CPU:0
Variable/read: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
Variable_1: (VariableV2): /job:localhost/replica:0/task:0/device:CPU:0
Variable_1/Assign: (Assign): /job:localhost/replica:0/task:0/device:CPU:0
Variable_1/read: (Identity): /job:localhost/replica:0/task:0/device:CPU:0
random_normal_1/RandomStandardNormal: (RandomStandardNormal): /job:localhost/replica:0/task:0/device:CPU:0
random_normal_1/mul: (Mul): /job:localhost/replica:0/task:0/device:CPU:0
random_normal_1: (AddV2): /job:localhost/replica:0/task:0/device:CPU:0
mul_1: (Mul): /

KeyboardInterrupt: 